In [ ]:
%%capture
import warnings
warnings.filterwarnings("ignore")

import calitp_data_analysis.magics

import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
import _new_ct_report_utils as utils
from update_vars import GTFS_DATA_DICT, analysis_month, file_name 

# Maps
import branca
import branca.colormap as cm
from shared_utils import webmap_utils
import geopandas as gpd

# Display
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify

In [ ]:
import google.auth
credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [ ]:
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas

@cache
def gcs_pandas():
    return GCSPandas()

In [ ]:
# district = '07-Los Angeles / Ventura'

In [ ]:
%%capture_parameters
district

In [ ]:
GCS_PATH = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/"

In [ ]:
# Files for webmaps
boundary_file = f"district_{district}_boundary"
transit_routes_file = f"district_{district}_transit_routes"
shn_file = f"district_{district}_shn"
transit_shn_file = f"district_{district}_transit_routes_shn"

In [ ]:
transit_route_shs_gdf, transit_route_shs_table, district_int = (
    utils.final_transit_route_shs_outputs(15, district)
)

In [ ]:
fct_monthly_routes_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet"

In [ ]:
operator_summary_url = f"{GCS_PATH}{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"

In [ ]:
operator_df = gcs_pandas().read_parquet(
    operator_summary_url,
    filters=[
        ("Caltrans District", "==", district),
        ("Date", "==", pd.Timestamp(analysis_month)),
        ("Day Type", "==", "Weekday")
    ],
)

In [ ]:
fct_monthly_route_df = gpd.read_parquet(fct_monthly_routes_url,
                                        filters=[[("Caltrans District", "==", district)]],
                                        storage_options={"token": credentials.token})

In [ ]:
fct_monthly_route_df = utils.prep_gdf(fct_monthly_route_df)

In [ ]:
district_gdf = utils.load_ct_district(district_int)

In [ ]:
shn_gdf =  utils.load_buffered_shn_map(district_int)

# {district}
These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

## District Overview

In [ ]:
try:
    operator_df2 = utils.create_summary_table(operator_df)
except:
    pass

In [ ]:
try:
    display(
    GT(operator_df2
        )
        .tab_header(title=f"District {district} GTFS summary stats")
    )
except:
    pass

## Routes within the District

In [ ]:
color_map = cm.linear.Spectral_11.scale()

In [ ]:
try:
    color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=fct_monthly_route_df.Number.max()
)
except:
    pass

In [ ]:
try:
    district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
    overwrite = True
)
except:
    pass

In [ ]:
try:
    transit_routes = webmap_utils.set_state_export(
    fct_monthly_route_df,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map,
    overwrite = True
    
)
except:
    pass

In [ ]:
try:
    webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")
except:
    pass

In [ ]:
try:
    webmap_utils.display_spa_map(transit_routes["spa_link"])
except:
    pass

## Transit Routes on the State Highway Network
**Only transit routes that have 15% or more if its length on one or more State Highway Network routes are included**

In [ ]:
color_map2 = cm.linear.RdYlBu_11.scale()

In [ ]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [ ]:
try:
    shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network',
    overwrite = True
)
except:
    pass

In [ ]:
try:
    transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
    overwrite = True
    
)
except:
    pass

In [ ]:
try:
    webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")
except:
    pass

In [ ]:
try:
    webmap_utils.display_spa_map(transit_shn_map["spa_link"])
except:
    pass

In [ ]:
try:
    display(GT(
    transit_route_shs_table.sort_values(
        by=[
            "Analysis Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
))
except:
    pass

## GTFS Stats by Operator

In [ ]:
try:
    gtfs_table = utils.create_operator_table(operator_df)
except:
    pass

In [ ]:
try:
    display((
    GT(gtfs_table.sort_values("Daily Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table.columns
            if c not in ["Operator"]
        ]
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics for Weekday by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table.columns
        ],
        align="center",
    )
))
except:
    pass